In [1]:
# Import dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
import sklearn as skl
import pandas as pd
import tensorflow as tf

In [45]:
# Import Hg dataset
Hg_D = pd.read_csv('../Data/Hg_D2.csv')
Hg_D.head()

,Unnamed: 0,Hg0_pM,HgT_pM,DMHg_pM,MMHg_pM,PRES,TEMP,PSAL,DOXY,NITRAT,NITRIT,SILCAT,PHSPHT
0,0,0.00,0.49,0.01,0.19,20.0,17.630000,35.183333,255.766667,7.495,0.600,2.63,0.965
1,1,0.17,0.69,0.01,0.00,30.0,17.493333,35.193333,252.666667,7.780,0.620,2.10,1.050
2,2,0.09,0.51,0.00,0.07,45.0,17.293333,35.190000,247.133333,7.420,0.600,1.27,1.310
3,3,0.03,0.98,0.03,0.06,80.0,14.186667,34.956667,23.033333,17.740,3.140,21.42,2.635
4,4,0.07,0.44,0.10,0.09,236.0,12.230000,34.870000,6.266667,14.785,8.955,31.28,2.540


In [46]:
# Drop Unnamed column
Hg_D.drop(columns=['Unnamed: 0','Hg0_pM','HgT_pM','DMHg_pM','NITRIT','SILCAT','PHSPHT'], inplace=True)
Hg_D.head()

,MMHg_pM,PRES,TEMP,PSAL,DOXY,NITRAT
0,0.19,20.0,17.630000,35.183333,255.766667,7.495
1,0.00,30.0,17.493333,35.193333,252.666667,7.780
2,0.07,45.0,17.293333,35.190000,247.133333,7.420
3,0.06,80.0,14.186667,34.956667,23.033333,17.740
4,0.09,236.0,12.230000,34.870000,6.266667,14.785


In [47]:
Hg_D = Hg_D.dropna()

In [48]:
Hg_D.shape

(509, 6)

In [49]:
# Split our preprocessed data into our features and target arrays
X = Hg_D.drop(['MMHg_pM'], axis=1)
y = Hg_D['MMHg_pM']

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=5)

In [50]:
# Create scaler instance
X_scaler = skl.preprocessing.StandardScaler()

# Fit the scaler
X_scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [51]:
# Create a method that creates a new Sequential model with hyperparameter options
def create_model(hp):
    nn_model = tf.keras.models.Sequential()

    # Allow kerastuner to decide which activation function to use in hidden layers
    activation = hp.Choice('activation',['relu','tanh','sigmoid'])
    
    # Allow kerastuner to decide number of neurons in first layer
    nn_model.add(tf.keras.layers.Dense(units=hp.Int('first_units',
        min_value=1,
        max_value=10,
        step=2), activation=activation, input_dim=5))

    # Allow kerastuner to decide number of hidden layers and neurons in hidden layers
    for i in range(hp.Int('num_layers', 1, 6)):
        nn_model.add(tf.keras.layers.Dense(units=hp.Int('units_' + str(i),
            min_value=1,
            max_value=10,
            step=2),
            activation=activation))
    
    nn_model.add(tf.keras.layers.Dense(units=1, activation="relu"))

    # Compile the model
    nn_model.compile(loss="binary_crossentropy", optimizer='adam', metrics=["accuracy"])
    
    return nn_model

In [52]:
# Import the kerastuner library
import kerastuner as kt

tuner = kt.Hyperband(
    create_model,
    objective="val_accuracy",
    max_epochs=20,
    hyperband_iterations=2)

INFO:tensorflow:Reloading Oracle from existing project .\untitled_project\oracle.json
INFO:tensorflow:Reloading Tuner from .\untitled_project\tuner0.json


In [53]:
# Run the kerastuner search for best hyperparameters
tuner.search(X_train_scaled,y_train,epochs=20,validation_data=(X_test_scaled,y_test))

INFO:tensorflow:Oracle triggered exit


In [54]:
# Get best model hyperparameters
best_hyper = tuner.get_best_hyperparameters(1)[0]
best_hyper.values

{'activation': 'sigmoid',
 'first_units': 3,
 'num_layers': 5,
 'units_0': 1,
 'units_1': 7,
 'units_2': 3,
 'units_3': 9,
 'units_4': 9,
 'tuner/epochs': 3,
 'tuner/initial_epoch': 0,
 'tuner/bracket': 2,
 'tuner/round': 0}

In [55]:
# Evaluate best model against full test data
best_model = tuner.get_best_models(1)[0]
model_loss, model_accuracy = best_model.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

4/4 - 0s - loss: 0.2856 - accuracy: 0.0781
Loss: 0.285562127828598, Accuracy: 0.078125
